In [1]:
try:
    from altinn3 import *
    from skatteetaten_api import get_access_token, base64_decode_response, decode_dokument
    import requests
    import base64
    import xmltodict
    import xml.dom.minidom
    from pathlib import Path
except ImportError as e:
    print("Mangler en eller avhengighet, installer dem via pip, se requierments.txt fil for detaljer")
    raise ImportError(e)


/Users/m93645/skattemeldingen/venv/lib/python3.12/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
def hent_gjeldende_skattemelding_og_sjekk_foraandsfastsettingstype(orgnr, inntektsaar):
    url_gjeldende = f'{base_url}/api/skattemelding/v2/{inntektsaar}/{orgnr}'
    r_gjeldende = s.get(url_gjeldende)

    if r_gjeldende:
        sme_og_naering_respons = xmltodict.parse(r_gjeldende.text)
        skattemelding_base64 = sme_og_naering_respons["skattemeldingOgNaeringsspesifikasjonforespoerselResponse"]["dokumenter"]["skattemeldingdokument"]
        decoded_sme_xml = decode_dokument(skattemelding_base64)
        sme_xml = xml.dom.minidom.parseString(decoded_sme_xml["content"]).toprettyxml()
        sme_dict = xmltodict.parse(sme_xml)
        if "gjelderForhaandsfastsetting" in sme_dict["skattemelding"]:
            forhaandsfastsetting =  sme_dict["skattemelding"]["gjelderForhaandsfastsetting"]['innsendingsformat']['forhaandsfastsettingsformattype']
        else:
            forhaandsfastsetting = "Parten har publisert skattemelding, men ikke klargjort for foråndsfastsetting, vil følge ordniært fastsettings løp"
        return forhaandsfastsetting
    else:
        return (r_gjeldende.status_code, r_gjeldende.text)


In [3]:
inntektsaar = "2024"
base_url = "https://idporten-api-sbstest.sits.no"
orgnr = "313074293" # bruk TENOR til å finne en part uten fastsetting av skattemeldingen og hvem som er daglig leder for orgnr (15859797943)


In [4]:
idporten_header = get_access_token()
s = requests.Session()
s.headers = dict(idporten_header)


The token is good, expires at 13:44:21 (3600 seconds from now)


127.0.0.1 - - [22/Mar/2024 12:44:21] "GET /token?code=mLMU21nOGhXzCLL0s-knbA.NgLABFTtUfiXqrbl36z4pA&iss=https%3A%2F%2Ftest.idporten.no&state=UtqOqtNj6XCtaydsGXOj0w HTTP/1.1" 200 -


In [6]:
#Hent gjeldende skattemelding, sjekker om den er klar, og om den har forhåndsfastsetting informasjon
sme_status = hent_gjeldende_skattemelding_og_sjekk_foraandsfastsettingstype(orgnr, inntektsaar)
print(f"Orgnr har følgende status: {sme_status} ")


Orgnr har følgende status: (403, 'Skattemeldingen er ikke tilgjengelig ennå. Prøv igjen senere.') 


In [7]:
url_klargjoer_forhaandsfastsetting = f"{base_url}/api/skattemelding/v2/klargjoerforhaandsfastsetting/{inntektsaar}/{orgnr}"


idporten_header["content-type"] = "application/json"
r_klargjoer = requests.post(url_klargjoer_forhaandsfastsetting, headers=idporten_header)
if r_klargjoer and ("application/json" in r_klargjoer.headers["content-type"]): 
    print(r_klargjoer.text)
else:
    print(r_klargjoer)
    

{"status":"OK"}


In [8]:
r_klargjoer.text

'{"status":"OK"}'

In [ ]:
sme_status = hent_gjeldende_skattemelding_og_sjekk_foraandsfastsettingstype(orgnr, inntektsaar)
print(f"Skattemeldingen er publisert, og foråhndsfastsettings type: {sme_status} ")

